In [ ]:
import os
import sys
import json
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
sys.path.insert(0, '../')
import utils.misc as workspace
from SkelPointNet import SkelPointNet 
from DataUtil import PCDataset, EllipsoidPcDataset
import FileRW as rw
import DistFunc as DF

In [ ]:
EXP_NAME = "full-5000-pskel-500"
experiment_dir = os.path.join("../experiments/", EXP_NAME)
split_file = 'val_split.txt'
checkpoint = 'latest'

with open(os.path.join(experiment_dir, "specs.json"), "r") as f:
    specs = json.load(f)

In [ ]:
data_dir = specs["DataSource"]
point_num = specs["InputPointNum"]
skelpoint_num = specs["SkelPointNum"]
to_normalize = specs["Normalize"]
# Assume Training/Test split file (given as cmd line arg) will be present in the experiment dir
pc_list_file = os.path.join(experiment_dir, split_file)
gpu = "0"
model_skel = SkelPointNet(
    num_skel_points=skelpoint_num, input_channels=0, use_xyz=True
)

if torch.cuda.is_available():
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu
    print("GPU Number:", torch.cuda.device_count(), "GPUs!")
    model_skel.cuda()
    model_skel.eval()

# Load the saved model
model_epoch = workspace.load_model_checkpoint(
    experiment_dir, checkpoint, model_skel
)
print(f"Evaluating model on using checkpoint={checkpoint} and epoch={model_epoch}.")

In [ ]:
# load data and evaluate
pc_list = rw.load_data_id(pc_list_file)

eval_data = PCDataset(pc_list, data_dir, point_num, to_normalize)
data_loader = DataLoader(
    dataset=eval_data, batch_size=1, shuffle=False, drop_last=False
)

overall_loss = 0
for _, batch_data in enumerate(tqdm(data_loader)):
    batch_id, batch_pc = batch_data
    batch_id = batch_id
    batch_pc = batch_pc.cuda().float()
    with torch.no_grad():
        skel_xyz, skel_r, _ = model_skel(batch_pc, compute_graph=False)
        loss = model_skel.get_sampling_loss(batch_pc, skel_xyz, skel_r)        
        overall_loss += loss.item()
overall_loss /= len(data_loader)

print(overall_loss)


In [ ]:
def compute_metrics_skeletal(batch_id, input_xyz, skel_xyz):
    """
    input_xyz : the g.t srep points
    skel_xyz : predicted skeletal points
    """
    batch_size = skel_xyz.size()[0]
    batch_id = batch_id.numpy()
    input_xyz_save = input_xyz.detach().cpu().numpy()
    skel_xyz_save = skel_xyz.detach().cpu().numpy()
    cd = 0
    hd = 0
    for i in range(batch_size):
        cd += DF.compute_pc_chamfer(input_xyz_save[i], skel_xyz_save[i])
        hd += DF.compute_pc_haussdorff(input_xyz_save[i], skel_xyz_save[i])
    cd /= batch_size
    hd /= batch_size
    return cd, hd

In [ ]:
label_list = rw.load_label_id(pc_list_file)
label_dir = "../data/ellipsoid_data_5000/pointcloud_srep/"

label_data = EllipsoidPcDataset(
    pc_list, label_list, 
    data_dir, label_dir,
    point_num, to_normalize
    )

label_loader = DataLoader(label_data, batch_size=1, shuffle=False, drop_last=False)

overall_cd = 0
overall_hd = 0
for _, batch_data,  in enumerate(tqdm(label_loader)):
    batch_id, batch_pc, batch_label = batch_data
    batch_id = batch_id
    batch_pc = batch_pc.cuda().float()
    with torch.no_grad():
        skel_xyz, skel_r, _ = model_skel(batch_pc, compute_graph=False)
        avg_cd_batch, avg_hd_batch = compute_metrics_skeletal(
                                        batch_id, 
                                        batch_label, 
                                        skel_xyz)
        overall_cd += avg_cd_batch
        overall_hd += avg_hd_batch

overall_cd /= len(data_loader)
overall_hd /= len(data_loader)
print(overall_cd, overall_hd)

